# 집계, 필터, 변환, 적용
- GroupBy 객체에는 그룹 데이터를 결합하기 전에 여러 유용한 연산을 효율적으로 구현하는 메서드가 있음
- aggregate() : 문자열, 함수, 리스트 등을 취해 한 번에 모든 집계를 계산

In [21]:
import random
import pandas as pd
import numpy as np

np.random.seed(5)

df = pd.DataFrame({'key' : ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1' : range(6),
                   'data2' : np.random.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,3
1,B,1,6
2,C,2,6
3,A,3,0
4,B,4,9
5,C,5,8


In [22]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     0    1.5   3
B       1    2.5   4     6    7.5   9
C       2    3.5   5     6    7.0   8

In [23]:
df.groupby('key').aggregate({'data1' : 'min', 'data2' : 'max'})

,data1,data2
key,,
A,0,3
B,1,9
C,2,8


- filter() : 속성을 기준으로 데이터를 걸러낼 수 있음
- 그룹이 필터링을 통과하는지 아닌지를 부울(bool) 값을 반환

In [39]:
def filter_func(x) :
    return x['data2'].std() > 2

print(df.groupby('key').std(), '\n')
print(df.groupby('key').filter(filter_func))

       data1     data2
key                   
A    2.12132  2.121320
B    2.12132  2.121320
C    2.12132  1.414214 

  key  data1  data2
0   A      0      3
1   B      1      6
3   A      3      0
4   B      4      9


- transform() : 재결합을 위해 전체 데이터의 변환된 버전을 반환

In [40]:
# 데이터에서 그룹별 평균값을 빼서 데이터를 중앙에 정렬하는 예제
df.groupby('key').transform(lambda x: x- x.mean())

,data1,data2
0,-1.5,1.5
1,-1.5,-1.5
2,-1.5,-1.0
3,1.5,-1.5
4,1.5,1.5
5,1.5,1.0


- apply() : 임의의 함수를 그룹 결과에 적용할 때 사용
- Pandas 객체 나 스칼라를 반환

In [50]:
# 첫 번째 열을 두 번째 열의 합계로 정규화한 예제
def norm_by_data2(x) :
    # x는 그룹 값을 가지는 DF
    x['data1'] /= x['data2'].sum()
    return x

df.groupby('key').apply(norm_by_data2)

,key,data1,data2
0,A,0.000000,3
1,B,0.066667,6
2,C,0.142857,6
3,A,1.000000,0
4,B,0.266667,9
5,C,0.357143,8


# 피벗 테이블
- 표 형태의 데이터로 작업하는 스프레드시티와 다른 프로그램에서 일반적으로 볼 수 있는 유사한 작업
- 입력값으로 간단한 열 단위의 데이터를 취하고 그 데이터에 대한 다차원 요약을 제공하는 2차원 테이블로 항목을 그룹핑함
- 근본적으로 GroupBy 집계의 다차원 버전이라고 생각!

In [51]:
import seaborn as sns
import numpy as np
import pandas as pd

planets = sns.load_dataset('planets')
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [53]:
planets.pivot_table('orbital_period', index='method', columns='number')

number,1,2,3,4,5,6,7
method,,,,,,,
Astrometry,631.180000,NaN,NaN,NaN,NaN,NaN,NaN
Eclipse Timing Variations,5821.166667,4216.883333,NaN,NaN,NaN,NaN,NaN
Imaging,140621.606250,NaN,NaN,73500.000000,NaN,NaN,NaN
Microlensing,3030.000000,3462.500000,NaN,NaN,NaN,NaN,NaN
Orbital Brightness Modulation,1.544929,0.291496,NaN,NaN,NaN,NaN,NaN
Pulsar Timing,18262.545353,NaN,63.338433,NaN,NaN,NaN,NaN
Pulsation Timing Variations,1170.000000,NaN,NaN,NaN,NaN,NaN,NaN
Radial Velocity,814.143794,959.022946,769.421724,695.769616,1045.993508,213.752357,NaN
Transit,12.301023,27.589652,22.187586,21.430775,30.584464,40.513600,119.217898


In [54]:
# 다단계 피벗 테이블
planets.pivot_table('orbital_period', ['method', 'year'], 'number')

number                                          1            2            3  \
method                        year                                            
Astrometry                    2010    1016.000000          NaN          NaN   
                              2013     246.360000          NaN          NaN   
Eclipse Timing Variations     2008            NaN  4544.000000          NaN   
                              2009   10220.000000          NaN          NaN   
                              2010            NaN  4228.525000          NaN   
                              2011    2900.000000  3878.125000          NaN   
                              2012    4343.500000          NaN          NaN   
Imaging                       2006  730000.000000          NaN          NaN   
                              2008  162140.000000          NaN          NaN   
                              2009    6885.116667          NaN          NaN   
                              2010            NaN          NaN          NaN   
                              2011   25018.750000          NaN          NaN   
Microlensing                  2005    3550.000000          NaN          NaN   
                              2006    3300.000000          NaN          NaN   
                              2008            NaN  3462.500000          NaN   
                              2010    1970.000000          NaN          NaN   
                              2011    2780.000000          NaN          NaN   
Orbital Brightness Modulation 2011            NaN     0.291496          NaN   
                              2013       1.544929          NaN          NaN   
Pulsar Timing                 1992            NaN          NaN    45.901950   
                              1994            NaN          NaN    98.211400   
                              2003   36525.000000          NaN          NaN   
                              2011       0.090706          NaN          NaN   
Pulsation Timing Variations   2007    1170.000000          NaN          NaN   
Radial Velocity               1989      83.888000          NaN          NaN   
                              1995       4.230785          NaN          NaN   
                              1996     306.167300          NaN  1078.000000   
                              1997      39.845000          NaN          NaN   
                              1998     442.100000    22.778902          NaN   
                              1999     229.170579  1491.596667          NaN   
                              2000     398.827586   878.467760          NaN   
                              2001     608.295270   349.700000  2391.000000   
                              2002     923.011456   572.128333  1224.730000   
                              2003     521.038743  1194.108000  1262.322500   
                              2004     562.830650   499.656800          NaN   
                              2005     478.080739  1526.500000   425.136928   
                              2006     943.195885  1178.400000    96.239250   
                              2007     499.341019   637.100000   453.000000   
                              2008     786.520456   753.524958  1047.791433   
                              2009    1358.676622   591.641913  2507.800087   
                              2010     415.020744  1522.644545  1050.300000   
                              2011     822.018312   950.318303    90.954711   
                              2012    1623.623202  1911.498889   862.642500   
                              2013     670.544883   142.400410   212.410333   
                              2014     990.196500  1904.260000   123.000000   
Transit                       2002       1.211919          NaN          NaN   
                              2004       2.653957          NaN          NaN   
                              2006       3.374686          NaN          NaN   
                              2007       3.020398 